In [56]:
import numpy as np 
import pandas as pd
import random
import keras
import torch
import tensorflow as tf
import optuna
from optuna import Trial
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from sklearn.metrics import log_loss
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
from sklearn import metrics

In [30]:
data = pd.read_excel("/Users/ziyuewang/Desktop/Inference Analytics/complain data/User complain input_top11.xlsx")

In [31]:
data["common complains"] = data["common complains"].map(lambda x: x.split("\n"))

In [32]:
data = data.explode("common complains")

In [33]:
data.head()

,common complains,specialities
0,Chest pain or discomfort (angina),Cardiology
0,Shortness of breath,Cardiology
0,Rapid or irregular heartbeats (arrhythmias),Cardiology
0,Heart failure,Cardiology
0,High blood pressure (hypertension),Cardiology


In [34]:
#data.to_excel("11 specialities_explode.xlsx")

In [35]:
new_data_dummy = pd.get_dummies(data.specialities)

In [36]:
new_data_dummy

,Cardiology,Dermatology,Internal Medicine,Neurology,Obstetrics,Obstetrics and Gynecology (OBGYN),Ophthalmology,Optometry,Pediatric Cardiology,Pediatric Dermatology,Primary Care
0,1,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
10,0,0,0,0,0,0,0,1,0,0,0
10,0,0,0,0,0,0,0,1,0,0,0
10,0,0,0,0,0,0,0,1,0,0,0
10,0,0,0,0,0,0,0,1,0,0,0


In [37]:
data = pd.concat([data,new_data_dummy],axis=1)

In [38]:
data = data.drop("specialities",axis=1)

In [39]:
data.head()

,common complains,Cardiology,Dermatology,Internal Medicine,Neurology,Obstetrics,Obstetrics and Gynecology (OBGYN),Ophthalmology,Optometry,Pediatric Cardiology,Pediatric Dermatology,Primary Care
0,Chest pain or discomfort (angina),1,0,0,0,0,0,0,0,0,0,0
0,Shortness of breath,1,0,0,0,0,0,0,0,0,0,0
0,Rapid or irregular heartbeats (arrhythmias),1,0,0,0,0,0,0,0,0,0,0
0,Heart failure,1,0,0,0,0,0,0,0,0,0,0
0,High blood pressure (hypertension),1,0,0,0,0,0,0,0,0,0,0


In [41]:
data.columns[1:]

Index(['Cardiology', 'Dermatology', 'Internal Medicine', 'Neurology',
       'Obstetrics', 'Obstetrics and Gynecology (OBGYN)', 'Ophthalmology',
       'Optometry', 'Pediatric Cardiology', 'Pediatric Dermatology',
       'Primary Care'],
      dtype='object')

In [18]:
X = data["common complains"]

In [42]:
specialities_labels = data[data.columns[1:]]

In [19]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [24]:
sentence_embedding = model.encode(X.values)

In [82]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(sentence_embedding, specialities_labels, test_size=0.3, random_state=42)

In [75]:
def objective(trial):
    keras.backend.clear_session()
    train_x, valid_x, train_y, valid_y = train_test_split(x_train, y_train, train_size=0.8, test_size=0.2,
                                                                random_state=42)
    #optimum number of hidden layers
    n_layers = trial.suggest_int('n_layers', 1, 3)
    model = keras.Sequential()
    for i in range(n_layers):
        #optimum number of hidden nodes
        num_hidden = trial.suggest_int(f'n_units_l{i}', 48, len(sentence_embedding[0]), log=True)
        #optimum activation function
        model.add(keras.layers.Dense(num_hidden, input_shape=(len(sentence_embedding[0]),),
                               activation=trial.suggest_categorical(f'activation{i}', ['relu', 'linear','swish'])))
        #optimum dropout value
        model.add(keras.layers.Dropout(rate = trial.suggest_float(f'dropout{i}', 0.0, 0.6))) 
    model.add(keras.layers.Dense(11,activation=tf.keras.activations.sigmoid)) #output Layer
    val_ds = (valid_x,valid_y)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=1,min_lr=1e-05,verbose=0)
    early_stoping = EarlyStopping(monitor="val_loss",min_delta=0,patience=5,verbose=0,mode="auto", baseline=None,restore_best_weights=True)
    model.compile(loss='binary_crossentropy',metrics='categorical_crossentropy', optimizer='Adam')
    #optimum batch size
    histroy = model.fit(train_x,train_y, validation_data=val_ds,epochs=30,callbacks=[reduce_lr,early_stoping],verbose=0,
                       batch_size=trial.suggest_int('size', 8, 128))
    return min(histroy.history['val_loss'])

In [76]:
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=50, timeout=1200)
    print("Number of finished trials: {}".format(len(study.trials)))
    print("Best trial:")
    trial = study.best_trial
    print("  Value: {}".format(trial.value))

[I 2023-02-12 11:07:18,857] A new study created in memory with name: no-name-21f3a890-5aee-4c77-b5aa-7fdba477fb6c
[I 2023-02-12 11:07:22,078] Trial 0 finished with value: 0.18274544179439545 and parameters: {'n_layers': 1, 'n_units_l0': 503, 'activation0': 'relu', 'dropout0': 0.24719261550579022, 'size': 40}. Best is trial 0 with value: 0.18274544179439545.
[I 2023-02-12 11:07:24,019] Trial 1 finished with value: 0.29155340790748596 and parameters: {'n_layers': 2, 'n_units_l0': 568, 'activation0': 'swish', 'dropout0': 0.5171165229656651, 'n_units_l1': 546, 'activation1': 'relu', 'dropout1': 0.3191537230731771, 'size': 118}. Best is trial 0 with value: 0.18274544179439545.
[I 2023-02-12 11:07:27,042] Trial 2 finished with value: 0.4127403795719147 and parameters: {'n_layers': 1, 'n_units_l0': 89, 'activation0': 'relu', 'dropout0': 0.5462005388021194, 'size': 102}. Best is trial 0 with value: 0.18274544179439545.
[I 2023-02-12 11:07:29,852] Trial 3 finished with value: 0.3075878024101257

[I 2023-02-12 11:08:21,759] Trial 24 finished with value: 0.18753819167613983 and parameters: {'n_layers': 2, 'n_units_l0': 189, 'activation0': 'linear', 'dropout0': 0.080900163280017, 'n_units_l1': 48, 'activation1': 'linear', 'dropout1': 0.22546972203966864, 'size': 32}. Best is trial 12 with value: 0.16334299743175507.
[I 2023-02-12 11:08:24,093] Trial 25 finished with value: 0.15930889546871185 and parameters: {'n_layers': 3, 'n_units_l0': 404, 'activation0': 'linear', 'dropout0': 0.00027060300753695143, 'n_units_l1': 109, 'activation1': 'relu', 'dropout1': 0.13890294472591902, 'n_units_l2': 147, 'activation2': 'relu', 'dropout2': 0.014257357486020639, 'size': 17}. Best is trial 25 with value: 0.15930889546871185.
[I 2023-02-12 11:08:25,578] Trial 26 finished with value: 0.33610987663269043 and parameters: {'n_layers': 3, 'n_units_l0': 365, 'activation0': 'linear', 'dropout0': 0.05771787822058421, 'n_units_l1': 107, 'activation1': 'relu', 'dropout1': 0.1455667602118753, 'n_units_l2

[I 2023-02-12 11:09:12,492] Trial 46 finished with value: 0.1596813052892685 and parameters: {'n_layers': 1, 'n_units_l0': 455, 'activation0': 'linear', 'dropout0': 0.06905476307744957, 'size': 22}. Best is trial 41 with value: 0.15399551391601562.
[I 2023-02-12 11:09:15,651] Trial 47 finished with value: 0.17181110382080078 and parameters: {'n_layers': 1, 'n_units_l0': 386, 'activation0': 'linear', 'dropout0': 0.025694584665453022, 'size': 31}. Best is trial 41 with value: 0.15399551391601562.
[I 2023-02-12 11:09:18,498] Trial 48 finished with value: 0.25041767954826355 and parameters: {'n_layers': 1, 'n_units_l0': 330, 'activation0': 'linear', 'dropout0': 0.21747759477218548, 'size': 97}. Best is trial 41 with value: 0.15399551391601562.
[I 2023-02-12 11:09:21,724] Trial 49 finished with value: 0.1679384559392929 and parameters: {'n_layers': 1, 'n_units_l0': 354, 'activation0': 'linear', 'dropout0': 0.0911320509881188, 'size': 23}. Best is trial 41 with value: 0.15399551391601562.


Number of finished trials: 50
Best trial:
  Value: 0.15399551391601562


In [77]:
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

  Params: 
    n_layers: 2
    n_units_l0: 459
    activation0: linear
    dropout0: 0.09056134729351364
    n_units_l1: 78
    activation1: swish
    dropout1: 0.14154633177645481
    size: 8


In [83]:
def wider_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(459,input_shape=(len(sentence_embedding[0]),),activation=tf.keras.activations.relu))
    model.add(keras.layers.Dropout(0.09056134729351364))
    model.add(keras.layers.Dense(11,activation=tf.keras.activations.sigmoid))
    return model

In [85]:
skf = KFold(n_splits=5, shuffle=True, random_state=42)
Final_Subbmission = []
val_loss_print = []
i=1
for train_index, test_index in skf.split(x_train,y_train):
    keras.backend.clear_session()
    print('#################')
    print(i)
    print('#################')
    X_train, X_test = x_train[train_index], x_train[test_index]
    train_y, test_y = y_train.iloc[train_index], y_train.iloc[test_index]
    model = wider_model()
    val_ds = (X_test,test_y)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=1,min_lr=1e-05,verbose=1)
    early_stoping = EarlyStopping(monitor="val_loss",min_delta=0,patience=5,verbose=1,mode="auto", baseline=None,restore_best_weights=True)
    model.compile(loss='binary_crossentropy',metrics='categorical_crossentropy', optimizer='Adam')
    histroy = model.fit(X_train,train_y, validation_data=val_ds,epochs=200,callbacks=[reduce_lr,early_stoping],verbose=1,batch_size=34)
    print(min(histroy.history['val_loss']))
    val_loss_print.append(min(histroy.history['val_loss']))
    Test_seq_pred = model.predict(x_test)
    Final_Subbmission.append(Test_seq_pred)
    i=i+1

#################
1
#################
Epoch 1/200
3/3 [==============================] - 1s 83ms/step - loss: 0.6872 - categorical_crossentropy: 2.3738 - val_loss: 0.6464 - val_categorical_crossentropy: 2.3504
Epoch 2/200
3/3 [==============================] - 0s 22ms/step - loss: 0.6319 - categorical_crossentropy: 2.2938 - val_loss: 0.5952 - val_categorical_crossentropy: 2.3113
Epoch 3/200
3/3 [==============================] - 0s 23ms/step - loss: 0.5746 - categorical_crossentropy: 2.2130 - val_loss: 0.5378 - val_categorical_crossentropy: 2.2688
Epoch 4/200
3/3 [==============================] - 0s 24ms/step - loss: 0.5095 - categorical_crossentropy: 2.1342 - val_loss: 0.4754 - val_categorical_crossentropy: 2.2229
Epoch 5/200
3/3 [==============================] - 0s 116ms/step - loss: 0.4434 - categorical_crossentropy: 2.0562 - val_loss: 0.4135 - val_categorical_crossentropy: 2.1750
Epoch 6/200
3/3 [==============================] - 0s 24ms/step - loss: 0.3806 - categorical_crossent

3/3 [==============================] - 0s 24ms/step - loss: 0.0633 - categorical_crossentropy: 0.2195 - val_loss: 0.1595 - val_categorical_crossentropy: 1.1188

Epoch 00038: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 39/200
3/3 [==============================] - 0s 25ms/step - loss: 0.0596 - categorical_crossentropy: 0.2019 - val_loss: 0.1595 - val_categorical_crossentropy: 1.1188
Epoch 40/200
3/3 [==============================] - 0s 27ms/step - loss: 0.0623 - categorical_crossentropy: 0.2161 - val_loss: 0.1595 - val_categorical_crossentropy: 1.1189
Restoring model weights from the end of the best epoch.
Epoch 00040: early stopping
0.15940071642398834
#################
3
#################
Epoch 1/200
3/3 [==============================] - 0s 83ms/step - loss: 0.6853 - categorical_crossentropy: 2.3986 - val_loss: 0.6450 - val_categorical_crossentropy: 2.3696
Epoch 2/200
3/3 [==============================] - 0s 22ms/step - loss: 0.6283 - categorical_crossentropy: 2.3259 -

Epoch 43/200
3/3 [==============================] - 0s 25ms/step - loss: 0.0612 - categorical_crossentropy: 0.2317 - val_loss: 0.1823 - val_categorical_crossentropy: 1.3414
Epoch 44/200
3/3 [==============================] - 0s 23ms/step - loss: 0.0553 - categorical_crossentropy: 0.1940 - val_loss: 0.1822 - val_categorical_crossentropy: 1.3414
Epoch 45/200
3/3 [==============================] - 0s 23ms/step - loss: 0.0571 - categorical_crossentropy: 0.2096 - val_loss: 0.1822 - val_categorical_crossentropy: 1.3414
Epoch 46/200
3/3 [==============================] - 0s 22ms/step - loss: 0.0633 - categorical_crossentropy: 0.2392 - val_loss: 0.1822 - val_categorical_crossentropy: 1.3415
Epoch 47/200
3/3 [==============================] - 0s 22ms/step - loss: 0.0549 - categorical_crossentropy: 0.1791 - val_loss: 0.1822 - val_categorical_crossentropy: 1.3415
Epoch 48/200
3/3 [==============================] - 0s 24ms/step - loss: 0.0626 - categorical_crossentropy: 0.2373 - val_loss: 0.1822 -

3/3 [==============================] - 0s 102ms/step - loss: 0.0658 - categorical_crossentropy: 0.2344 - val_loss: 0.1480 - val_categorical_crossentropy: 0.9240
Epoch 81/200
3/3 [==============================] - 0s 26ms/step - loss: 0.0628 - categorical_crossentropy: 0.2174 - val_loss: 0.1480 - val_categorical_crossentropy: 0.9239
Epoch 82/200
3/3 [==============================] - 0s 25ms/step - loss: 0.0609 - categorical_crossentropy: 0.2037 - val_loss: 0.1480 - val_categorical_crossentropy: 0.9237
Epoch 83/200
3/3 [==============================] - 0s 26ms/step - loss: 0.0631 - categorical_crossentropy: 0.2130 - val_loss: 0.1479 - val_categorical_crossentropy: 0.9236
Epoch 84/200
3/3 [==============================] - 0s 25ms/step - loss: 0.0652 - categorical_crossentropy: 0.2263 - val_loss: 0.1479 - val_categorical_crossentropy: 0.9234
Epoch 85/200
3/3 [==============================] - 0s 26ms/step - loss: 0.0646 - categorical_crossentropy: 0.2254 - val_loss: 0.1479 - val_categor

3/3 [==============================] - 0s 23ms/step - loss: 0.0579 - categorical_crossentropy: 0.1793 - val_loss: 0.1471 - val_categorical_crossentropy: 0.9186
Epoch 175/200
3/3 [==============================] - 0s 24ms/step - loss: 0.0661 - categorical_crossentropy: 0.2473 - val_loss: 0.1471 - val_categorical_crossentropy: 0.9185
Epoch 176/200
3/3 [==============================] - 0s 23ms/step - loss: 0.0588 - categorical_crossentropy: 0.1960 - val_loss: 0.1471 - val_categorical_crossentropy: 0.9185
Epoch 177/200
3/3 [==============================] - 0s 23ms/step - loss: 0.0594 - categorical_crossentropy: 0.1942 - val_loss: 0.1471 - val_categorical_crossentropy: 0.9185
Epoch 178/200
3/3 [==============================] - 0s 23ms/step - loss: 0.0623 - categorical_crossentropy: 0.2225 - val_loss: 0.1471 - val_categorical_crossentropy: 0.9185
Epoch 179/200
3/3 [==============================] - 0s 23ms/step - loss: 0.0616 - categorical_crossentropy: 0.2076 - val_loss: 0.1471 - val_cat

Epoch 17/200
3/3 [==============================] - 0s 25ms/step - loss: 0.1619 - categorical_crossentropy: 0.9341 - val_loss: 0.2078 - val_categorical_crossentropy: 1.3962
Epoch 18/200
3/3 [==============================] - 0s 23ms/step - loss: 0.1532 - categorical_crossentropy: 0.8676 - val_loss: 0.2027 - val_categorical_crossentropy: 1.3579
Epoch 19/200
3/3 [==============================] - 0s 22ms/step - loss: 0.1431 - categorical_crossentropy: 0.7628 - val_loss: 0.1987 - val_categorical_crossentropy: 1.3280
Epoch 20/200
3/3 [==============================] - 0s 23ms/step - loss: 0.1344 - categorical_crossentropy: 0.7020 - val_loss: 0.1946 - val_categorical_crossentropy: 1.2963
Epoch 21/200
3/3 [==============================] - 0s 89ms/step - loss: 0.1357 - categorical_crossentropy: 0.7207 - val_loss: 0.1899 - val_categorical_crossentropy: 1.2570
Epoch 22/200
3/3 [==============================] - 0s 23ms/step - loss: 0.1242 - categorical_crossentropy: 0.6054 - val_loss: 0.1847 -

In [87]:
test_prob =np.mean(Final_Subbmission,0)
test_prob = pd.DataFrame(test_prob)
test_prob.columns = specialities_labels.columns

In [88]:
y_test1= y_test.reset_index(drop=True)
print("precision: {:.2f} ".format( metrics.average_precision_score(y_test1, test_prob, average='weighted')))

precision: 0.77 


In [92]:
test_prob

,Cardiology,Dermatology,Internal Medicine,Neurology,Obstetrics,Obstetrics and Gynecology (OBGYN),Ophthalmology,Optometry,Pediatric Cardiology,Pediatric Dermatology,Primary Care
0,0.033538,0.058927,0.058335,0.716033,0.013640,0.018765,0.053051,0.047617,0.023775,0.007814,0.027823
1,0.001995,0.003762,0.005888,0.063716,0.003783,0.011201,0.163877,0.200203,0.007098,0.003783,0.003913
2,0.002918,0.013443,0.002398,0.006187,0.000301,0.003011,0.549651,0.013646,0.001975,0.018217,0.001097
3,0.001543,0.575407,0.011175,0.018798,0.005517,0.017264,0.004405,0.005444,0.002961,0.171417,0.019914
4,0.039048,0.009366,0.025215,0.561007,0.002206,0.003529,0.093352,0.009197,0.009129,0.008846,0.006568
5,0.034844,0.012249,0.008247,0.006757,0.170622,0.634945,0.027707,0.018917,0.019547,0.052981,0.018870
6,0.187277,0.002218,0.017119,0.272148,0.008269,0.021935,0.010136,0.050219,0.096232,0.002369,0.024557
7,0.005995,0.622884,0.006937,0.018469,0.001990,0.012093,0.028813,0.002994,0.002844,0.217218,0.006386
8,0.011525,0.015204,0.537930,0.005446,0.062644,0.005426,0.003542,0.015826,0.018893,0.019864,0.469686
9,0.856081,0.002387,0.010975,0.009090,0.004723,0.009906,0.005514,0.007411,0.111084,0.005296,0.013802
